In [2]:
# importamos librerías
import pandas as pd
from datetime import datetime

# **Carga del archivo**

In [3]:
# cargamos el archivo con los datos
df_raw = pd.read_parquet("../data/reservas_hoteles.parquet")

In [4]:
# info del dataframe con los datos
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


In [5]:
# Hacemos copia del dataframe para trabajar
df = df_raw.copy()

# Cambio columnas con fechas a datetime

In [6]:
# Las columnas de fecha_reserva, inicio_estancia y final_estancia deberían estar en formato fecha. Hacemos una función para transformarlas.
def trans_fecha(lista_col, dataframe):
    for col in lista_col:
        dataframe[col] = pd.to_datetime(dataframe[col], errors="coerce")
    return dataframe

In [7]:
df = trans_fecha(["fecha_reserva", "inicio_estancia", "final_estancia"], df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

# Fechas estancia

In [8]:
# miramos valores únicos de fechas de inicio y final de estancia porque las necesitaremos para buscar eventos en la API
ini_estancia =df["inicio_estancia"].unique()
ini_estancia

<DatetimeArray>
['2025-03-01 00:00:00', 'NaT']
Length: 2, dtype: datetime64[ns]

In [9]:
fin_estancia = df["final_estancia"].unique()
fin_estancia

<DatetimeArray>
['2025-03-02 00:00:00', 'NaT']
Length: 2, dtype: datetime64[ns]

Como las fechas son las mismas, podríamos rellenar toda la columna con el mismo valor.

In [10]:
# Rellenamos las columnas del df con las fechas.
df["inicio_estancia"] = ini_estancia[0]
df["final_estancia"] = fin_estancia[0]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15098 non-null  datetime64[ns]
 8   final_estancia   15098 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

In [12]:
df.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
9294,5cb1f95e-13c3-48de-b3eb-317d9f6fd221,e283ccec-e2b2-48fe-9e25-863e978a7523,Margarita,Hierro,margarita.hierro@example.com,False,2025-02-06,2025-03-01,2025-03-02,14,415.13,Hotel Sol y Luna,4.0,Madrid
14148,53d6a499-6a5a-4a25-9ec5-c002b89d7793,87e0ac59-99ff-4374-987e-4c94ffe4a546,Inés,Codina,inés.codina@example.com,False,2025-02-06,2025-03-01,2025-03-02,38,92.01,Hotel Maravilla Real,1.0,Madrid
2140,ae51580f-cae5-4043-909a-c5345dc998fa,c4e93473-6d71-48b6-82d1-13730bc4e18b,Calista,Miguel,calista.miguel@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,379.38,Hotel Brisas del Mar,3.0,Madrid
13896,1d49502e-72fc-4536-8702-c64b82f39850,c7bc7c4e-d951-4ced-ad08-07432beece82,Trini,Vallés,trini.vallés@example.com,False,2025-02-05,2025-03-01,2025-03-02,15,459.45,Hotel Palacio Imperial,2.0,Madrid
1002,6b91ec40-85ad-404a-ae4f-7352a1256789,7e4d17fd-e8c3-4b73-b38a-3808d5ae6b99,Iris,Bustos,iris.bustos@example.com,False,2025-02-02,2025-03-01,2025-03-02,15,235.89,Hotel Vista Alegre,2.0,Madrid


# Ciudades

In [13]:
ciudades = df["ciudad"].unique()
ciudades

array(['', 'Madrid'], dtype=object)

In [14]:
# Rellenamos toda la columna con Madrid.
df["ciudad"] = "Madrid"
df.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
523,3780edb0-dcf4-432a-9994-badfca89b9f8,87b652fc-6b02-4dab-8115-c96299d8c6a0,Iker,Noguera,iker.noguera@example.com,False,2025-02-12,2025-03-01,2025-03-02,47,374.62,Hotel Torre Dorada,4.0,Madrid
3937,6037ad85-e8ff-4b7f-a95b-6bf74d31d200,3fa41334-f344-45c8-835a-5838f92fedde,Bartolomé,Carro,bartolomé.carro@example.com,False,2025-02-04,2025-03-01,2025-03-02,19,261.38,Hotel Sol y Luna,3.0,Madrid
7899,0463f8dd-f69d-4217-982b-4b4e2cbcd716,ddb5434f-d33b-43ba-b7d4-fe8b6d3f7dc0,Jose,Valdés,jose.valdés@example.com,False,2025-02-02,2025-03-01,2025-03-02,13,133.48,Palacio del Sol,4.0,Madrid
8129,19c00c83-bf3d-4b8d-b9c8-f715efda1d56,2f4e8ebc-1fa3-4056-a5b6-43e54a9be3af,Octavio,Cabañas,octavio.cabañas@example.com,False,2025-02-08,2025-03-01,2025-03-02,28,74.62,Hotel Camino del Sol,4.0,Madrid
4639,4ee81f41-5b83-4ce7-bb4d-5aa37a52ff56,18643114-8a4b-4d33-84a6-f6ef8572b37b,Elvira,Perez,elvira.perez@example.com,False,2025-02-01,2025-03-01,2025-03-02,13,127.22,Hotel Camino del Sol,3.0,Madrid


In [15]:
df["ciudad"].unique()


array(['Madrid'], dtype=object)

# Duplicados

In [16]:
duplicados = df.duplicated()
duplicados.sum()

np.int64(98)

In [17]:
# Hay 98 filas totalmente duplicadas, las eliminamos
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    9828 non-null   datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     9776 non-null   float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        9828 non-null   float64       
 13  ciudad           15000 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64(2), 

# **Transformación**

De cara a cargar los datos en la base de datos vamos a generar unas tablas a partir del df.

In [18]:
# Podemos tener id_hotel duplicados porque más de un cliente puede reservar en el mismo hotel y podemos tener id_cliente duplicados porque el mismo cliente puede haber hecho varias reservas
# lo que no podemos tener repetido es el id_reserva. Lo comprobamos
reservas = df["id_reserva"].duplicated().sum()
reservas

np.int64(0)

## Hoteles

In [19]:
# Cantidad de id_hotel
df["id_hotel"].nunique()

29

In [20]:
# Cantidad nombres hotel
df["nombre_hotel"].nunique()

20

Hay discrepancia en la cantidad de valores únicos. Vamos a generar dos tablas de hoteles, una con los hoteles propios y otra con los de la competencia.

### Hoteles propios

In [21]:
df_propios = df[["id_hotel", "nombre_hotel" , "estrellas", "ciudad", "competencia"]][df["competencia"]==False]
df_propios.sample(5)

,id_hotel,nombre_hotel,estrellas,ciudad,competencia
9650,6,Hotel Monte Verde,4.0,Madrid,False
10148,28,Hotel Palacio Imperial,4.0,Madrid,False
4312,15,Hotel Camino del Sol,1.0,Madrid,False
7308,40,Hotel Brisas del Mar,3.0,Madrid,False
6885,19,Hotel Camino del Sol,4.0,Madrid,False


In [22]:
df_propios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9828 entries, 2 to 14999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      9828 non-null   int64  
 1   nombre_hotel  9828 non-null   object 
 2   estrellas     9828 non-null   float64
 3   ciudad        9828 non-null   object 
 4   competencia   9828 non-null   bool   
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 393.5+ KB


In [23]:
# Comprobamos si hay hoteles con más de un id asignado
conteo_hoteles = df_propios.groupby("nombre_hotel")["id_hotel"].nunique().reset_index()
conteo_hoteles

,nombre_hotel,id_hotel
0,Gran Hotel Madrid,19
1,Hotel Brisas del Mar,19
2,Hotel Camino del Sol,19
3,Hotel Costa Azul,19
4,Hotel Encanto Real,19
5,Hotel Jardines del Rey,19
6,Hotel Las Estrellas,19
7,Hotel Los Almendros,19
8,Hotel Luz de Madrid,19
9,Hotel Maravilla Real,19


Todos los hoteles tienen 19 id asignados, lo que no puede ser correcto. Vamos a asociar un único id_hotel por cada nombre_hotel

In [24]:
nombrespropios = list(df_propios["nombre_hotel"].unique())
nombrespropios

['Hotel Monte Verde',
 'Hotel Brisas del Mar',
 'Hotel Camino del Sol',
 'Hotel Puerta del Cielo',
 'Hotel Encanto Real',
 'Palacio del Sol',
 'Hotel Jardines del Rey',
 'Hotel Las Estrellas',
 'Gran Hotel Madrid',
 'Hotel Torre Dorada',
 'Hotel Palacio Imperial',
 'Hotel Luz de Madrid',
 'Hotel Los Almendros',
 'Hotel Sol y Luna',
 'Hotel Mirador Real',
 'Hotel Rincón Sereno',
 'Hotel Vista Alegre',
 'Hotel Costa Azul',
 'Hotel Maravilla Real']

In [25]:
# Crear una lista de números seriales según la cantidad de hoteles propios
idspropios = list(range(1, len(nombrespropios)+1))
# Asignar números seriales a cada elemento. Lo hacemos como un diccionario para poder hacer un mapeo en el df_propios
hoteles_propios = dict(zip(nombrespropios, idspropios))



In [26]:
df_propios["id_hotel"] = df_propios["nombre_hotel"].map(hoteles_propios)
df_propios.sample(5)

,id_hotel,nombre_hotel,estrellas,ciudad,competencia
12504,10,Hotel Torre Dorada,3.0,Madrid,False
14127,5,Hotel Encanto Real,1.0,Madrid,False
5041,7,Hotel Jardines del Rey,4.0,Madrid,False
10624,12,Hotel Luz de Madrid,2.0,Madrid,False
14044,18,Hotel Costa Azul,1.0,Madrid,False


La columna estrellas se supone que se refiere a la clasificación del hotel, por lo que cada hotel debería tener una clasificación única. Lo comprobamos

In [27]:
conteo_estrellas = df_propios.groupby("nombre_hotel")["estrellas"].nunique().reset_index()
conteo_estrellas

,nombre_hotel,estrellas
0,Gran Hotel Madrid,5
1,Hotel Brisas del Mar,5
2,Hotel Camino del Sol,5
3,Hotel Costa Azul,5
4,Hotel Encanto Real,5
5,Hotel Jardines del Rey,5
6,Hotel Las Estrellas,5
7,Hotel Los Almendros,5
8,Hotel Luz de Madrid,5
9,Hotel Maravilla Real,5


Vemos cada hotel tiene 5 clasificaciones distintas, lo que no tendría sentido. Vamos a suponer que se trata de la valoración de los clientes, por lo que tomaremos el valor medio de cada hotel.

In [28]:
# Cambiamos el nombre de la columna
df_propios.rename(columns={"estrellas":"valoracion"}, inplace=True)

In [29]:
# Calculamos la valoracion media por cada nombre de hotel y redondeamos a 1 decimal
df_valoracion_propios = round(df_propios.groupby("nombre_hotel")["valoracion"].mean().reset_index(),1)
df_valoracion_propios

,nombre_hotel,valoracion
0,Gran Hotel Madrid,3.1
1,Hotel Brisas del Mar,3.1
2,Hotel Camino del Sol,3.1
3,Hotel Costa Azul,3.1
4,Hotel Encanto Real,3.0
5,Hotel Jardines del Rey,2.9
6,Hotel Las Estrellas,2.9
7,Hotel Los Almendros,3.0
8,Hotel Luz de Madrid,3.1
9,Hotel Maravilla Real,3.0


In [30]:
def generar_mapa(col1, col2, dataframe):
    """
    Genera un diccionario a partir de los valores de dos columnas de un DataFrame, generando pares clave:valor por cada fila.

    Parámetros:
    col1 (str): Nombre de la primera columna que se usará como claves del diccionario.
    col2 (str): Nombre de la segunda columna que se usará como valores del diccionario.
    dataframe (pandas.DataFrame): El DataFrame que contiene las columnas a mapear.

    Retorna:
    dict: Un diccionario donde las claves son los valores de la columna col1 y los valores son los valores de la columna col2.
    """
    lista_col1 = []
    lista_col2 = []
    for i in dataframe[col1]:
        lista_col1.append(i)
    for f in dataframe[col2]:
        lista_col2.append(f)

    mapa = dict(zip(lista_col1, lista_col2))
    return mapa

In [31]:
# generamos un diccionario con los nombres de los hoteles y su valoración media
valoracion_propios = generar_mapa("nombre_hotel", "valoracion", df_valoracion_propios)

In [32]:
# Usamos el diccionario para mapear el df de hoteles propios y corregir la columna de estrellas
df_propios["valoracion"] = df_propios["nombre_hotel"].map(valoracion_propios)
df_propios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9828 entries, 2 to 14999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      9828 non-null   int64  
 1   nombre_hotel  9828 non-null   object 
 2   valoracion    9828 non-null   float64
 3   ciudad        9828 non-null   object 
 4   competencia   9828 non-null   bool   
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 393.5+ KB


### Hoteles competencia

In [33]:
competencia = df[["id_hotel", "nombre_hotel", "estrellas", "ciudad", "competencia"]][df["competencia"]==True]
competencia.sample(5)

,id_hotel,nombre_hotel,estrellas,ciudad,competencia
9256,181,,NaN,Madrid,True
11796,186,,NaN,Madrid,True
7633,135,,NaN,Madrid,True
2854,117,,NaN,Madrid,True
93,113,,NaN,Madrid,True


In [34]:
# Cambiamos el nombre de la columna
competencia.rename(columns={"estrellas":"valoracion"}, inplace=True)

In [35]:
competencia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5172 entries, 0 to 14998
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      5172 non-null   int64  
 1   nombre_hotel  5172 non-null   object 
 2   valoracion    0 non-null      float64
 3   ciudad        5172 non-null   object 
 4   competencia   5172 non-null   bool   
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 207.1+ KB


In [36]:
ids_competencia = (competencia["id_hotel"].unique()).tolist()

Los datos que faltan los extraemos por scrapeo

In [37]:
info_competencia = pd.read_pickle("hoteles_competencia.pkl")
info_competencia

,nombre,valoracion,precio,fecha_reserva
0,ibis Styles Madrid Prado,4.7,179.0,2025-02-20
1,ibis budget Madrid Calle 30,4.4,106.0,2025-02-20
2,ibis Madrid Centro las Ventas,4.5,181.0,2025-02-20
3,ibis budget Madrid Centro las Ventas,4.3,119.0,2025-02-20
4,ibis budget Madrid Vallecas,4.3,102.0,2025-02-20
5,ibis Madrid Aeropuerto Barajas,4.4,129.0,2025-02-20
6,ibis Madrid Alcorcon Tresaguas,4.4,95.0,2025-02-20
7,ibis budget Madrid Aeropuerto,4.0,97.0,2025-02-20
8,ibis Madrid Alcobendas,4.4,89.0,2025-02-20
9,ibis budget Madrid Alcorcon Móstoles,4.5,77.0,2025-02-20


In [38]:
info_competencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre         10 non-null     object        
 1   valoracion     10 non-null     float64       
 2   precio         10 non-null     float64       
 3   fecha_reserva  10 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 452.0+ bytes


In [39]:
info_competencia["id_hotel"] = ids_competencia
info_competencia["ciudad"] = "Madrid"
info_competencia

,nombre,valoracion,precio,fecha_reserva,id_hotel,ciudad
0,ibis Styles Madrid Prado,4.7,179.0,2025-02-20,113,Madrid
1,ibis budget Madrid Calle 30,4.4,106.0,2025-02-20,194,Madrid
2,ibis Madrid Centro las Ventas,4.5,181.0,2025-02-20,131,Madrid
3,ibis budget Madrid Centro las Ventas,4.3,119.0,2025-02-20,114,Madrid
4,ibis budget Madrid Vallecas,4.3,102.0,2025-02-20,103,Madrid
5,ibis Madrid Aeropuerto Barajas,4.4,129.0,2025-02-20,181,Madrid
6,ibis Madrid Alcorcon Tresaguas,4.4,95.0,2025-02-20,128,Madrid
7,ibis budget Madrid Aeropuerto,4.0,97.0,2025-02-20,186,Madrid
8,ibis Madrid Alcobendas,4.4,89.0,2025-02-20,135,Madrid
9,ibis budget Madrid Alcorcon Móstoles,4.5,77.0,2025-02-20,117,Madrid


In [40]:
info_competencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre         10 non-null     object        
 1   valoracion     10 non-null     float64       
 2   precio         10 non-null     float64       
 3   fecha_reserva  10 non-null     datetime64[ns]
 4   id_hotel       10 non-null     int64         
 5   ciudad         10 non-null     object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 612.0+ bytes


In [41]:
df_competencia = info_competencia[["id_hotel", "nombre", "valoracion", "ciudad"]]
df_competencia["competencia"] = True
df_competencia.rename(columns={"nombre" : "nombre_hotel"}, inplace = True)
df_competencia

C:\Users\marta\AppData\Local\Temp\ipykernel_20092\244128482.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_competencia["competencia"] = True
C:\Users\marta\AppData\Local\Temp\ipykernel_20092\244128482.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_competencia.rename(columns={"nombre" : "nombre_hotel"}, inplace = True)


,id_hotel,nombre_hotel,valoracion,ciudad,competencia
0,113,ibis Styles Madrid Prado,4.7,Madrid,True
1,194,ibis budget Madrid Calle 30,4.4,Madrid,True
2,131,ibis Madrid Centro las Ventas,4.5,Madrid,True
3,114,ibis budget Madrid Centro las Ventas,4.3,Madrid,True
4,103,ibis budget Madrid Vallecas,4.3,Madrid,True
5,181,ibis Madrid Aeropuerto Barajas,4.4,Madrid,True
6,128,ibis Madrid Alcorcon Tresaguas,4.4,Madrid,True
7,186,ibis budget Madrid Aeropuerto,4.0,Madrid,True
8,135,ibis Madrid Alcobendas,4.4,Madrid,True
9,117,ibis budget Madrid Alcorcon Móstoles,4.5,Madrid,True


In [42]:
df_competencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      10 non-null     int64  
 1   nombre_hotel  10 non-null     object 
 2   valoracion    10 non-null     float64
 3   ciudad        10 non-null     object 
 4   competencia   10 non-null     bool   
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 462.0+ bytes


In [43]:
#unimos los dos dataframes de hoteles
df_hoteles = pd.concat([df_propios, df_competencia], axis = 0)
df_hoteles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9838 entries, 2 to 9
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      9838 non-null   int64  
 1   nombre_hotel  9838 non-null   object 
 2   valoracion    9838 non-null   float64
 3   ciudad        9838 non-null   object 
 4   competencia   9838 non-null   bool   
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 393.9+ KB


In [57]:
valoracion_hoteles = generar_mapa("nombre_hotel", "valoracion", df_hoteles)
nombres_id_hoteles = generar_mapa("nombre_hotel", "id_hotel", df_hoteles)
nombres_competencia = generar_mapa("id_hotel", "nombre_hotel", df_competencia)
precios_competencia = generar_mapa("nombre", "precio", info_competencia)

## Clientes

Como podemos tener dos clientes con el mismo nombre, tomaremos el mail de cliente como el valor único de cliente. Es posible que tengamos un mismo cliente que haya hecho vrias reservas, por lo que encontrar el mismo id_cliente varias veces tampoco indicaría error. Vamos a comprobar si el número de valores únicos de id_cliente se corresponde con el número de valores únicos de mail. De no ser así, podría indicar un error en la asignación de id_cliente que debemos corregir

In [45]:
df["id_cliente"].nunique()

14847

In [46]:
df["mail"].nunique()

14905

Vemos que hay más valores de mail que de id_cliente, lo que sugiere que más de un cliente tiene el mismo id. Vamos a corregir el id de la misma forma que hemos corregido los id_hotel.

In [47]:
df_clientes = df[["id_cliente", "nombre", "apellido", "mail"]]
df_clientes.sample(5)

,id_cliente,nombre,apellido,mail
11459,c82d7ec4-ad52-4c24-8a07-8ae05daaa7bd,Primitiva,Diez,primitiva.diez@example.com
3455,2a05614f-6051-4604-b8c0-60d2c0345add,Dolores,Ángel,dolores.ángel@example.com
7489,b4898556-3fa0-4476-a995-09e1e008d7b8,Luis,Morell,luis.morell@example.com
7049,9ad04ff0-5989-46a2-aa2e-6ef9badf9ff8,Isaac,Puig,isaac.puig@example.com
12403,182edb9a-b2b4-4fbf-803f-c6c07cdef107,Nazaret,Verdugo,nazaret.verdugo@example.com


In [48]:
mails=df_clientes["mail"].unique()

In [49]:
mailcliente=[]
idcliente=[]
for mail in enumerate(mails, start=1):
    mailcliente.append(mail[1])
    idcliente.append(f"cliente_{mail[0]}")
mail_cliente = dict(zip(mailcliente, idcliente))

In [50]:
df_clientes.loc[:, "id_cliente"] = df_clientes["mail"].map(mail_cliente)
df_clientes.sample(5)

,id_cliente,nombre,apellido,mail
13398,cliente_13328,Tecla,Maestre,tecla.maestre@example.com
13967,cliente_13885,Federico,Ribas,federico.ribas@example.com
7938,cliente_7908,Adela,Valle,adela.valle@example.com
4208,cliente_4203,Alondra,Torrijos,alondra.torrijos@example.com
11068,cliente_11020,Prudencio,Berrocal,prudencio.berrocal@example.com


## Reservas

In [51]:
# Renombramos la columna estrellas por valoracion
df.rename(columns = {"estrellas": "ESTRELLAS", "id_hotel" : "ID_HOTEL", "nombre_hotel": "NOMBRE_HOTEL", "precio_noche": "PRECIO"}, inplace =  True)

In [ ]:
# La fecha de reserva para los hoteles de la competencia es la del scrapeo. Lo actualizamos
df.loc[df["competencia"]==True, "fecha_reserva"] = info_competencia["fecha_reserva"][0]
df["id_cliente"] = df["mail"].map(mail_cliente)
df["id_hotel"] = df.apply(nombres_id_hoteles.get())

In [56]:
df.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,ID_HOTEL,PRECIO,NOMBRE_HOTEL,ESTRELLAS,ciudad
14730,b1a9a634-cdd4-47fd-8607-b6a3484dffe3,cliente_14640,Efraín,Moraleda,efraín.moraleda@example.com,False,2025-02-07,2025-03-01,2025-03-02,36,148.91,Hotel Rincón Sereno,4.0,Madrid
3265,de4773be-eb4d-438f-965b-c1cd6e44eb62,cliente_3261,David,Pedro,david.pedro@example.com,True,2025-02-20,2025-03-01,2025-03-02,117,NaN,,NaN,Madrid
3327,9e7b2510-6b4c-4403-87a3-2d107b01ff36,cliente_3323,Andrés,Blazquez,andrés.blazquez@example.com,False,2025-02-12,2025-03-01,2025-03-02,27,209.81,Hotel Brisas del Mar,3.0,Madrid
13785,04d309d7-325b-4086-864e-fcc630908f2c,cliente_4610,Reyes,Cruz,reyes.cruz@example.com,True,2025-02-20,2025-03-01,2025-03-02,114,NaN,,NaN,Madrid
3294,49cd29a1-94af-41f8-9bdd-a490fe9d6d1b,cliente_3290,Leire,Mateu,leire.mateu@example.com,False,2025-02-06,2025-03-01,2025-03-02,3,469.93,Hotel Sol y Luna,2.0,Madrid


In [53]:
#df["id_hotel"] = df["nombre_hotel"].map(hoteles_propios).fillna(df["nombre_hotel"])
#df["nombre_hotel"] = df["id_hotel"].map(nombres_competencia).fillna(df["id_hotel"])
#df["precio_noche"] = df["nombre_hotel"].map(precios_competencia).fillna(df["id_hotel"])
#df["valoracion"] = df["nombre_hotel"].map(valoracion_hoteles)
#df.info()


In [54]:
df.sample(10)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,ID_HOTEL,PRECIO,NOMBRE_HOTEL,ESTRELLAS,ciudad
6619,4ccfa33e-2137-4d0f-ba09-6fc2574dd1d1,cliente_6599,Américo,Feliu,américo.feliu@example.com,True,2025-02-20,2025-03-01,2025-03-02,131,NaN,,NaN,Madrid
13792,91650756-0162-46ef-84a0-a3cc74a7e63b,cliente_13711,Norberto,Leon,norberto.leon@example.com,False,2025-02-01,2025-03-01,2025-03-02,13,95.20,Hotel Sol y Luna,3.0,Madrid
14669,7e420ef3-f529-4fd1-9ef9-27cf721ac81e,cliente_14579,Maricela,Domínguez,maricela.domínguez@example.com,False,2025-02-04,2025-03-01,2025-03-02,40,115.36,Hotel Mirador Real,3.0,Madrid
9317,6876e11b-de51-4b57-9609-f38ff9b4d19c,cliente_9281,Mauricio,Polo,mauricio.polo@example.com,True,2025-02-20,2025-03-01,2025-03-02,114,NaN,,NaN,Madrid
150,7078d68a-f0a9-4e31-bde8-25b5247a1f38,cliente_151,Sebastian,Morera,sebastian.morera@example.com,False,2025-02-05,2025-03-01,2025-03-02,36,201.25,Hotel Luz de Madrid,2.0,Madrid
818,da1e2438-e512-42d5-9bab-b199aeb31a93,cliente_819,Cayetano,Sandoval,cayetano.sandoval@example.com,False,2025-02-11,2025-03-01,2025-03-02,14,379.81,Palacio del Sol,1.0,Madrid
11861,bfccf87b-99cc-4905-b8f6-b5718f2e4d4e,cliente_11809,Isa,Andrés,isa.andrés@example.com,True,2025-02-20,2025-03-01,2025-03-02,181,NaN,,NaN,Madrid
1956,68e6e343-e67f-464e-87d9-33250a45db27,cliente_1957,Rufino,Zapata,rufino.zapata@example.com,False,2025-02-08,2025-03-01,2025-03-02,43,364.82,Palacio del Sol,3.0,Madrid
6324,e7a619b6-861d-46de-a3f7-184997c27562,cliente_6306,Angelina,Buendía,angelina.buendía@example.com,False,2025-02-08,2025-03-01,2025-03-02,3,338.70,Hotel Palacio Imperial,3.0,Madrid
3871,48463a65-5663-4567-a818-ad9d55181a66,cliente_3867,Alfonso,Coloma,alfonso.coloma@example.com,False,2025-02-12,2025-03-01,2025-03-02,33,437.66,Palacio del Sol,1.0,Madrid


In [55]:
df_reservas = df[["id_reserva", "fecha_reserva", "inicio_estancia", "final_estancia", "precio_noche", "id_cliente", "id_hotel"]]

KeyError: "['precio_noche', 'id_hotel'] not in index"

In [ ]:
df_reservas.sample(5)

,id_reserva,fecha_reserva,inicio_estancia,final_estancia,precio_noche,id_cliente,id_hotel
7024,568b287d-bcc0-46e7-b306-f8e1abc963ad,2025-02-10,2025-03-01,2025-03-02,18.0,cliente_7003,18.0
8660,4e467a80-2336-4ac3-9a8b-69a6b96729a6,2025-02-12,2025-03-01,2025-03-02,3.0,cliente_8628,3.0
8780,7ba670ba-426f-4577-94c1-72b0b13745e8,2025-02-04,2025-03-01,2025-03-02,9.0,cliente_8747,9.0
14973,de52ac47-ce39-4bde-96d1-1480b5199bf8,2025-02-03,2025-03-01,2025-03-02,19.0,cliente_14879,19.0
13811,647a9a69-ddd2-46b7-8e61-5ee6e2e9abba,2025-02-20,2025-03-01,2025-03-02,106.0,cliente_13730,194.0
